In [1]:
import random
from functools import partial
from itertools import product
from pathlib import Path

import torch
from torch import Generator, eye, tensor, use_deterministic_algorithms, zeros_like
from torch.nn.functional import cross_entropy, one_hot

from karpathy_series.makemore.bigrams import NGram
from karpathy_series.makemore.components.function.loss import CrossEntropyLoss
from karpathy_series.makemore.components.generation import NGramNetGenerator
from karpathy_series.makemore.components.models.model import Model
from karpathy_series.makemore.components.networks.mlp import MPLNet
from karpathy_series.makemore.data import read_data
from karpathy_series.makemore.encoding.character import CharacterEncoder, CharacterSet, StringEncoder
from karpathy_series.makemore.training.data import DataSplit, TrainingSequencer
from karpathy_series.makemore.training.learning import LearningRecord

random.seed(42)

%matplotlib inline
use_deterministic_algorithms(True)
rand_generator = Generator()
seed = 2147483647


def create_generator():
    return rand_generator.manual_seed(seed)

In [2]:
data_path = Path("../../data/names.txt")
words = read_data(data_path)
char_set = CharacterSet.from_words(words)
char_encoder = CharacterEncoder.from_charset(char_set)
string_encoder = StringEncoder(char_encoder)

context_size = 3
embedding_dims = 10
hidden_dims = 64

n_gram_gen = partial(NGram.generate, context_size, char_set.pad)
ts = TrainingSequencer(string_encoder, char_encoder, n_gram_gen)
data_split = DataSplit.split(words, 80, 10, 10)
learning_sequence = ts.training_sequence(data_split.training, 32, True)
validation_set = ts.training_set(data_split.validation)

In [3]:
g = rand_generator.manual_seed(seed)
mpl_net = MPLNet.init(1, char_encoder.size, embedding_dims, context_size, hidden_dims, False, g)
model = Model(mpl_net, CrossEntropyLoss())
generator = NGramNetGenerator(char_set, string_encoder, char_encoder, mpl_net, context_size)
learning_record = LearningRecord()
print(f"{mpl_net.size()} parameters")
print(f"{mpl_net.describe()} parameters")

4137 parameters
An MLP model: Sequence [Embedding [27, 10], Flatten last 2 dims, Linear [30, 64] with bias, BatchNorm1d [64], Functional <built-in method tanh of type object at 0x7f98d2b3ecc0>, Linear [64, 27] with bias] parameters


In [4]:
# utility function we will use later when comparing manual gradients to PyTorch gradients
def cmp(s, dt, t):
    ex = torch.all(dt == t.grad).item()
    app = torch.allclose(dt, t.grad)
    maxdiff = (dt - t.grad).abs().max().item()
    print(f"{s:15s} | exact: {str(ex):5s} | approximate: {str(app):5s} | maxdiff: {maxdiff}")

In [5]:
layers = mpl_net.layers

C = layers[0].embedding

# Layer 1
W1, b1 = layers[2].weight, layers[2].bias

# Layer 2
W2, b2 = layers[5].weight, layers[5].bias

# BatchNorm parameters
bngain, bnbias = layers[3].gamma, layers[3].beta

In [6]:
Xb, Yb = next(learning_sequence())
n = Xb.shape[0]
c = len(char_set.complete)

emb = C[Xb]  # embed the characters into vectors
embcat = emb.view(emb.shape[0], -1)  # concatenate the vectors

# Linear layer 1
hprebn = embcat @ W1 + b1  # hidden layer pre-activation

# BatchNorm layer
bnmeani = 1 / n * hprebn.sum(0, keepdim=True)
bndiff = hprebn - bnmeani
bndiff2 = bndiff**2
bnvar = 1 / (n - 1) * (bndiff2).sum(0, keepdim=True)  # note: Bessel's correction (dividing by n-1, not n)
bnvar_inv = (bnvar + 1e-5) ** -0.5
bnraw = bndiff * bnvar_inv
hpreact = bngain * bnraw + bnbias

# Non-linearity
h = torch.tanh(hpreact)  # hidden layer

# Linear layer 2
logits = h @ W2 + b2  # output layer

# cross entropy loss (same as F.cross_entropy(logits, Yb))
logit_maxes = logits.max(1, keepdim=True).values
norm_logits = logits - logit_maxes  # subtract max for numerical stability
counts = norm_logits.exp()
counts_sum = counts.sum(1, keepdims=True)
counts_sum_inv = counts_sum**-1  # if I use (1.0 / counts_sum) instead then I can't get backprop to be bit exact...
probs = counts * counts_sum_inv
logprobs = probs.log()
loss = -logprobs[range(n), Yb].mean()

intermediates = [
    logprobs,
    probs,
    counts,
    counts_sum,
    counts_sum_inv,
    norm_logits,
    logit_maxes,
    logits,
    h,
    hpreact,
    bnraw,
    bnvar_inv,
    bnvar,
    bndiff2,
    bndiff,
    hprebn,
    bnmeani,
    embcat,
    emb,
]

# PyTorch backward pass
for p in mpl_net.parameters():
    p.grad = None
for t in intermediates:
    t.retain_grad()
loss.backward()
loss

tensor(3.2965, grad_fn=<NegBackward0>)

In [7]:
# Exercise 1: backprop through the whole thing manually,
# backpropagating through exactly all of the variables
# as they are defined in the forward pass above, one by one

mask = zeros_like(logprobs)
mask[range(n), Yb] = 1
dlogprobs = -1 / n * mask
dprobs = dlogprobs / probs
dcounts_sum_inv = (dprobs * counts).sum(axis=1, keepdims=True)
dcounts_sum = -(counts_sum**-2) * dcounts_sum_inv
dcounts = counts_sum_inv * dprobs + dcounts_sum
dnorm_logits = norm_logits.exp() * dcounts
dlogit_maxes = -dnorm_logits.sum(axis=1, keepdims=True)

# This selects the max from each row, masking the gradient of the max
logit_max_selector = one_hot(logits.max(1).indices, num_classes=logits.shape[1])
dlogits = dnorm_logits + logit_max_selector * dlogit_maxes

dh = dlogits @ W2.T
dW2 = h.T @ dlogits
db2 = dlogits.sum(axis=0)
dhpreact = (1.0 - h**2) * dh

dbngain = (bnraw * dhpreact).sum(0)
dbnbias = dhpreact.sum(0)
dbnraw = bngain * dhpreact
dbnvar_inv = (bndiff * dbnraw).sum(0)
dbnvar = -0.5 * (bnvar + 1e-5) ** -1.5 * dbnvar_inv
dbndiff2 = 1 / (n - 1) * dbnvar
dbndiff = 2 * bndiff * dbndiff2 + bnvar_inv * dbnraw
dbnmeani = -dbndiff.sum(0)
dhprebn = dbndiff + (1 / n) * dbnmeani

dembcat = dhprebn @ W1.T
dW1 = embcat.T @ dhprebn
db1 = dhprebn.sum(axis=0)

demb = dembcat.view(dembcat.shape[0], context_size, embedding_dims)

# From lecture. I think this could be vectorized
dC = zeros_like(C)
for k, j in product(range(n), range(Xb.shape[1])):
    dC[Xb[k, j]] += demb[k, j]

cmp("logprobs", dlogprobs, logprobs)
cmp("probs", dprobs, probs)
cmp("counts_sum_inv", dcounts_sum_inv, counts_sum_inv)
cmp("counts_sum", dcounts_sum, counts_sum)
cmp("counts", dcounts, counts)
cmp("norm_logits", dnorm_logits, norm_logits)
cmp("logit_maxes", dlogit_maxes, logit_maxes)
cmp("logits", dlogits, logits)
cmp("h", dh, h)
cmp("W2", dW2, W2)
cmp("b2", db2, b2)
cmp("hpreact", dhpreact, hpreact)
cmp("bngain", dbngain, bngain)
cmp("bnbias", dbnbias, bnbias)
cmp("bnraw", dbnraw, bnraw)
cmp("bnvar_inv", dbnvar_inv, bnvar_inv)
cmp("bnvar", dbnvar, bnvar)
cmp("bndiff2", dbndiff2, bndiff2)
cmp("bndiff", dbndiff, bndiff)
cmp("bnmeani", dbnmeani, bnmeani)
cmp("hprebn", dhprebn, hprebn)
cmp("embcat", dembcat, embcat)
cmp("W1", dW1, W1)
cmp("b1", db1, b1)
cmp("emb", demb, emb)
cmp("C", dC, C)

logprobs        | exact: True  | approximate: True  | maxdiff: 0.0
probs           | exact: True  | approximate: True  | maxdiff: 0.0
counts_sum_inv  | exact: True  | approximate: True  | maxdiff: 0.0
counts_sum      | exact: True  | approximate: True  | maxdiff: 0.0
counts          | exact: True  | approximate: True  | maxdiff: 0.0
norm_logits     | exact: True  | approximate: True  | maxdiff: 0.0
logit_maxes     | exact: True  | approximate: True  | maxdiff: 0.0
logits          | exact: True  | approximate: True  | maxdiff: 0.0
h               | exact: True  | approximate: True  | maxdiff: 0.0
W2              | exact: True  | approximate: True  | maxdiff: 0.0
b2              | exact: True  | approximate: True  | maxdiff: 0.0
hpreact         | exact: False | approximate: True  | maxdiff: 9.094947017729282e-13
bngain          | exact: False | approximate: True  | maxdiff: 1.0913936421275139e-11
bnbias          | exact: False | approximate: True  | maxdiff: 2.9103830456733704e-11
bnraw 

In [8]:
# Exercise 2: backprop through cross_entropy but all in one go
# to complete this challenge look at the mathematical expression of the loss,
# take the derivative, simplify the expression, and just write it out
loss_fast = cross_entropy(logits, Yb)
print(loss_fast.item(), "diff:", (loss_fast - loss).item())

3.296450614929199 diff: 0.0


In [9]:
from torch.nn.functional import one_hot
from torch.testing import assert_close

x = tensor([[1, 2, 3, 4, 5, 6], [7, 8, 9, 10, 11, 12], [13, 14, 15, 16, 17, 18]])
y = tensor([1, 4, 0])
o = one_hot(y, num_classes=6)
assert_close(x[range(x.shape[0]), y], (o * x).sum(axis=1))

Yo = one_hot(Yb, num_classes=c)
loss_alt = -(Yo * (logits.softmax(1)).log()).sum(1).mean()
assert_close(loss, loss_alt)

In [10]:
# My version of a backward pass
counts = logits.softmax(1)
dlogits = 1 / n * counts * (one_hot(Yb, num_classes=c).unsqueeze(2) * (1 - eye(c, c) / counts.unsqueeze(2))).sum(1)

# Andrej's version from the lecture
# I should reconcile this with the above (though the above is apparently more precise)
_dlogits = counts.clone()
_dlogits[range(n), Yb] -= 1
_dlogits *= 1 / n

# I really need a better set of properties to reason algebraically with the one_hot
# The final factor of 1/n and initial `counts` are common
# consequently, the following must be equivalent:
#    X[range(n), Yb] -= 1
#    X *= (one_hot(Yb, ...).unsqueeze(2) * (1 - Id/X.unsqueeze(2))).sum(1)

cmp("logits", dlogits, logits)
cmp("logits", _dlogits, logits)
# Andrej can only get approximate to be true, with a maxdiff of 6e-9
# Mine agrees if we both use `* 1/n` rather than `/n`.

logits          | exact: False | approximate: True  | maxdiff: 1.2078089639544487e-09
logits          | exact: False | approximate: True  | maxdiff: 1.2078089639544487e-09


In [11]:
# Exercise 3: backprop through batchnorm but all in one go
# to complete this challenge look at the mathematical expression of the output of batchnorm,
# take the derivative w.r.t. its input, simplify the expression, and just write it out
# BatchNorm paper: https://arxiv.org/abs/1502.03167

# forward pass

# before:
# bnmeani = 1/n*hprebn.sum(0, keepdim=True)
# bndiff = hprebn - bnmeani
# bndiff2 = bndiff**2
# bnvar = 1/(n-1)*(bndiff2).sum(0, keepdim=True) # note: Bessel's correction (dividing by n-1, not n)
# bnvar_inv = (bnvar + 1e-5)**-0.5
# bnraw = bndiff * bnvar_inv
# hpreact = bngain * bnraw + bnbias

# now:
hprebn_std = torch.sqrt(hprebn.var(0, keepdim=True, unbiased=True) + 1e-5)
hprebn_norm = (hprebn - hprebn.mean(0, keepdim=True)) / hprebn_std
hpreact_fast = bngain * hprebn_norm + bnbias
print("max diff:", (hpreact_fast - hpreact).abs().max())

max diff: tensor(4.7684e-07, grad_fn=<MaxBackward1>)


In [12]:
# backward pass

# before we had:
# dbnraw = bngain * dhpreact
# dbndiff = bnvar_inv * dbnraw
# dbnvar_inv = (bndiff * dbnraw).sum(0, keepdim=True)
# dbnvar = (-0.5*(bnvar + 1e-5)**-1.5) * dbnvar_inv
# dbndiff2 = (1.0/(n-1))*torch.ones_like(bndiff2) * dbnvar
# dbndiff += (2*bndiff) * dbndiff2
# dhprebn = dbndiff.clone()
# dbnmeani = (-dbndiff).sum(0)
# dhprebn += 1.0/n * (torch.ones_like(hprebn) * dbnmeani)

# calculate dhprebn given dhpreact (i.e. backprop through the batchnorm)
# (you'll also need to use some of the variables from the forward pass up above)

dhprebn = (
    bngain
    * (dhpreact - ((1 / n + 1 / (n - 1) * hprebn_norm * hprebn_norm.unsqueeze(1)) * dhpreact).sum(1))
    / hprebn_std
)

cmp("hprebn", dhprebn, hprebn)

# Andrej's version
_dhprebn = bngain * bnvar_inv / n * (n * dhpreact - dhpreact.sum(0) - n / (n - 1) * bnraw * (dhpreact * bnraw).sum(0))
cmp("hprebn", _dhprebn, hprebn)

hprebn          | exact: False | approximate: True  | maxdiff: 1.0913936421275139e-11
hprebn          | exact: False | approximate: True  | maxdiff: 1.0913936421275139e-11


In [13]:
# Exercise 4: putting it all together!
# Train the MLP neural net with your own backward pass

# init
n_embd = 10  # the dimensionality of the character embedding vectors
n_hidden = 200  # the number of neurons in the hidden layer of the MLP
encoding_size = char_encoder.size

g = torch.Generator().manual_seed(2147483647)  # for reproducibility

mpl_net = MPLNet.init(1, encoding_size, n_embd, context_size, n_hidden, False, g)
layers = mpl_net.layers
C = layers[0].embedding
W1, b1 = layers[2].weight, layers[2].bias
W2, b2 = layers[5].weight, layers[5].bias
bngain, bnbias = layers[3].gamma, layers[3].beta

parameters = [C, W1, b1, W2, b2, bngain, bnbias]
print(sum(p.nelement() for p in parameters))  # number of parameters in total

# same optimization as last time
max_steps = 10_000
batch_size = 32
n = batch_size  # convenience
lossi = []

with torch.no_grad():
    Xtr, Ytr = ts.training_set(data_split.training)

# kick off optimization
for i in range(max_steps):
    # minibatch construct
    ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
    Xb, Yb = Xtr[ix], Ytr[ix]  # batch X,Y

    # forward pass
    emb = C[Xb]  # embed the characters into vectors
    embcat = emb.view(emb.shape[0], -1)  # concatenate the vectors
    # Linear layer
    hprebn = embcat @ W1 + b1  # hidden layer pre-activation
    # BatchNorm layer
    # -------------------------------------------------------------
    bnmean = hprebn.mean(0, keepdim=True)
    bnvar = hprebn.var(0, keepdim=True, unbiased=True)
    bnvar_inv = (bnvar + 1e-5) ** -0.5
    bnraw = (hprebn - bnmean) * bnvar_inv
    hpreact = bngain * bnraw + bnbias
    # -------------------------------------------------------------
    # Non-linearity
    h = torch.tanh(hpreact)  # hidden layer
    logits = h @ W2 + b2  # output layer
    loss = cross_entropy(logits, Yb)  # loss function

    # backward pass
    for p in parameters:
        p.grad = None
    # loss.backward() # use this for correctness comparisons, delete it later!

    counts = logits.softmax(1)
    dlogits = (
        -1
        / n
        * (one_hot(Yb, num_classes=c).unsqueeze(2) * (eye(c, c) / counts.unsqueeze(2) - 1) * counts.unsqueeze(1)).sum(1)
    )
    dh = dlogits @ W2.T
    dW2 = h.T @ dlogits
    db2 = dlogits.sum(axis=0)
    dhpreact = (1.0 - h**2) * dh

    dbngain = (bnraw * dhpreact).sum(0)
    dbnbias = dhpreact.sum(0)
    hprebn_std = torch.sqrt(hprebn.var(0, keepdim=True, unbiased=True) + 1e-5)
    hprebn_norm = (hprebn - hprebn.mean(0, keepdim=True)) / hprebn_std
    dhprebn = (
        bngain
        * hprebn_std**-1
        * (dhpreact - ((1 / n + 1 / (n - 1) * hprebn_norm * hprebn_norm.unsqueeze(1)) * dhpreact).sum(1))
    )

    dembcat = dhprebn @ W1.T
    dW1 = embcat.T @ dhprebn
    db1 = dhprebn.sum(axis=0)
    demb = dembcat.view(dembcat.shape[0], context_size, embedding_dims)

    # From lecture. I think this could be vectorized
    dC = zeros_like(C)
    for k, j in product(range(n), range(Xb.shape[1])):
        dC[Xb[k, j]] += demb[k, j]

    grads = [dC, dW1, db1, dW2, db2, dbngain, dbnbias]
    # -----------------

    # update
    lr = 0.1 if i < 100000 else 0.01  # step learning rate decay
    for p, grad in zip(parameters, grads):
        p.data += -lr * grad  # new way of swole doge TODO: enable

    # track stats
    if i % 10000 == 0:  # print every once in a while
        print(f"{i:7d}/{max_steps:7d}: {loss.item():.4f}")
    lossi.append(loss.log10().item())

12297
      0/  10000: 3.2968


In [14]:
# useful for checking your gradients
# for p,g in zip(parameters, grads):
#   cmp(str(tuple(p.shape)), g, p)

In [15]:
# calibrate the batch norm at the end of training

with torch.no_grad():
    # pass the training set through
    emb = C[Xtr]
    embcat = emb.view(emb.shape[0], -1)
    hpreact = embcat @ W1 + b1
    # measure the mean/std over the entire training set
    bnmean = hpreact.mean(0, keepdim=True)
    bnvar = hpreact.var(0, keepdim=True, unbiased=True)

In [16]:
# evaluate train and val loss


@torch.no_grad()  # this decorator disables gradient tracking
def split_loss(split):
    x, y = {
        "train": ts.training_set(data_split.training),
        "val": ts.training_set(data_split.validation),
        "test": ts.training_set(data_split.development),
    }[split]
    emb = C[x]  # (N, block_size, n_embd)
    embcat = emb.view(emb.shape[0], -1)  # concat into (N, block_size * n_embd)
    hpreact = embcat @ W1 + b1
    hpreact = bngain * (hpreact - bnmean) * (bnvar + 1e-5) ** -0.5 + bnbias
    h = torch.tanh(hpreact)  # (N, n_hidden)
    logits = h @ W2 + b2  # (N, vocab_size)
    loss = cross_entropy(logits, y)
    print(split, loss.item())


split_loss("train")
split_loss("val")

train 2.2498579025268555
val 2.26698899269104


In [ ]:
# Andrej achieved:
# train 2.0718822479248047
# val 2.1162495613098145
#
# I got:
# train 2.068848133087158
# val 2.110271453857422

In [17]:
# sample from the model
g = torch.Generator().manual_seed(2147483647 + 10)

for _ in range(20):
    out = []
    context = [0] * context_size  # initialize with all ...
    while True:
        # forward pass
        emb = C[torch.tensor([context])]  # (1,block_size,d)
        embcat = emb.view(emb.shape[0], -1)  # concat into (N, block_size * n_embd)
        hpreact = embcat @ W1 + b1
        hpreact = bngain * (hpreact - bnmean) * (bnvar + 1e-5) ** -0.5 + bnbias
        h = torch.tanh(hpreact)  # (N, n_hidden)
        logits = h @ W2 + b2  # (N, vocab_size)
        # sample
        probs = logits.softmax(1)
        ix = torch.multinomial(probs, num_samples=1, generator=g).item()
        context = context[1:] + [ix]
        out.append(ix)
        if ix == 0:
            break

    print("".join(char_encoder.decode_or_raise(i) for i in out))

mora.
mmyanna.
elma.
hayla.
rensry.
brdraeg.
azeredielie.
maliona.
edelleananar.
kayzioh.
kamin.
shubergahiriel.
kindreelyn.
xethuorius.
macder.
yah.
faelsylay.
miston.
mahal.
calynnsun.
